# Test cases

This notebook set up and run the test cases in [McWilliams et al., 1997](https://doi.org/10.1017/S0022112096004375) using KPPLT-LF17 ([Li & Fox-Kemper, 2017](https://doi.org/10.1175/JPO-D-17-0085.1)) and SMCLT-H15 ([Harcourt, 2015](https://doi.org/10.1175/JPO-D-14-0046.1)).


In [1]:
import sys
import copy
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../gotmtool/.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='ADHOC', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/code
   gotmdir_data: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/data
  gotmdir_build: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/build
    gotmdir_exe: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/exe
    gotmdir_run: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/run
 gotmdir_figure: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/figure
   gotmdir_tool: /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 11.7 ms, sys: 35.1 ms, total: 46.8 ms
Wall time: 93.7 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/run/ADHOC/gotm.yaml'...
Done!


 ------------------------------------------------------------------------
 GOTM started on 2023/12/08 at 19:44:36
 ------------------------------------------------------------------------
    initialize_gotm
 ------------------------------------------------------------------------
        Reading configuration from: gotm.yaml
        configuring modules ....
    init_airsea_yaml
        done
    init_observations_yaml
    init_stokes_drift_yaml
        done
    init_turbulence_yaml
        done.
    init_cvmix_yaml
        done.
    init_meanflow_yaml
        done
    init_eqstate_yaml
        done.
 Your configuration has been written to /data1/scratch/qingli/project/adhoc/A2020_ADHOC/gotm/run/ADHOC/gotm.yaml.
STOP 0


Update the configuration

In [6]:
# setup
title = 'ADHOC'
depth = 90.0     # m
tau = 0.037      # N m^{-2}
Q0 = -5          # W m^{-2}
rho0 = 1000      # kg m^{-3}
alphaT = 2e-4    # degC^{-1}
betaS = 8e-4     # psu^{-1}

cfg['title'] = title
cfg['location']['name'] = 'idealized'
cfg['location']['longitude'] = 0.0
cfg['location']['depth'] = depth
cfg['time']['start'] = '2000-01-01 00:00:00'
cfg['time']['stop']  = '2000-01-05 00:00:00'
cfg['time']['dt']    = 60.0

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 60
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = m.environ['gotmdir_data']
cfg['temperature']['method'] = 'file'
cfg['temperature']['file'] = os.path.join(datadir, 'tprof_file_MSM97.dat')
cfg['salinity']['method'] = 'file'
cfg['salinity']['file'] = os.path.join(datadir, 'sprof_file_MSM97.dat')
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['heat']['constant_value'] = Q0
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['constant_value'] = tau
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['method'] = 'constant'
cfg['surface']['swr']['constant_value'] = 0.0
cfg['surface']['precip']['method'] = 'constant'
cfg['surface']['precip']['constant_value'] = 0.0

# Stokes drift
cfg['waves']['stokes_drift']['us']['method'] = 'exponential'
cfg['waves']['stokes_drift']['vs']['method'] = 'exponential'
cfg['waves']['stokes_drift']['dusdz']['method'] = 'us'
cfg['waves']['stokes_drift']['dvsdz']['method'] = 'vs'
cfg['waves']['stokes_drift']['exponential']['us0']['method'] = 'constant'
cfg['waves']['stokes_drift']['exponential']['vs0']['method'] = 'constant'
cfg['waves']['stokes_drift']['exponential']['vs0']['constant_value'] = 0.0
cfg['waves']['stokes_drift']['exponential']['ds']['method'] = 'constant'
cfg['waves']['stokes_drift']['exponential']['ds']['constant_value'] = 4.8

# EOS -- use linear
cfg['eq_state']['form'] = 'linear_custom'
cfg['eq_state']['linear']['T0'] = 20.0
cfg['eq_state']['linear']['S0'] = 35.0
cfg['eq_state']['linear']['dtr0'] = -alphaT*rho0
cfg['eq_state']['linear']['dsr0'] = betaS*rho0
cfg['physical_constants']['rho_0'] = rho0


## Create a list of configurations

In [7]:
# Forcing                                                                                                                                                               
cases = {
    'MSM97-LT':    {'lat': 45.0, 'us0':  0.068},
    'MSM97-ST':    {'lat': 45.0, 'us0':  0.0},
    'MSM97-LTf0':  {'lat': 0.0,  'us0':  0.068},
    'MSM97-STf0':  {'lat': 0.0,  'us0':  0.0},
}

# Resolution
dz = {'1m': 1.0, '10m': 10.0}

# Turbulence methods
turbmethods = [
    'SMCLT-H15',
    'KPP-CVMix',
    'KPPLT-LF17',
    ]

In [8]:
for casename in cases:
    print(casename)

MSM97-LT
MSM97-ST
MSM97-LTf0
MSM97-STf0


In [9]:
cfgs = []
labels = []

for res in dz:
    print('dz = {:s}'.format(res))
    nlev = int(depth/dz[res])
    cfg['grid']['nlev']  = nlev
    cfg['output']['gotm_out']['k1_stop'] = nlev+1
    cfg['output']['gotm_out']['k_stop'] = nlev
    for casename in cases:
        print(' - {:s}'.format(casename))
        cfg['location']['latitude'] = cases[casename]['lat']
        cfg['waves']['stokes_drift']['exponential']['us0']['constant_value'] = cases[casename]['us0']
        for turbmethod in turbmethods: 
            run_label = '{:s}_{:s}'.format(turbmethod, res)
            labels.append(os.path.join(casename, run_label))
            run_dir = os.path.join(m.environ['gotmdir_run'], m.name, casename, run_label)
            os.makedirs(run_dir, exist_ok=True)
            if turbmethod == 'SMCLT-H15':
                cfg['turbulence']['turb_method'] = 'second_order'
                cfg['turbulence']['tke_method'] = 'mellor_yamada'
                cfg['turbulence']['len_scale_method'] = 'mellor_yamada'
                cfg['turbulence']['scnd']['method'] =  'quasi_eq_h15'
                cfg['turbulence']['scnd']['scnd_coeff'] =  'kantha_clayson'
                cfg['turbulence']['turb_param']['length_lim'] = 'false'
                cfg['turbulence']['turb_param']['compute_c3'] = 'false'
                cfg['turbulence']['my']['e3'] = 5.0
                cfg['turbulence']['my']['e6'] = 6.0
            elif turbmethod == 'KPP-CVMix':
                cfg['turbulence']['turb_method'] = 'cvmix'
                cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
            elif turbmethod == 'KPPLT-LF17':
                cfg['turbulence']['turb_method'] = 'cvmix'
                cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'lf17'
            else:
                raise ValueError('Turbulence closure method \'{}\' not defined.'.format(turbmethod))
            cfgs.append(copy.deepcopy(cfg))

dz = 1m
 - MSM97-LT
 - MSM97-ST
 - MSM97-LTf0
 - MSM97-STf0
dz = 10m
 - MSM97-LT
 - MSM97-ST
 - MSM97-LTf0
 - MSM97-STf0


## Run the model

In [10]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=4)

CPU times: user 57.6 ms, sys: 37.7 ms, total: 95.2 ms
Wall time: 1.66 s
